In [1]:
#loading modules and  packages
import logging
import collections
import datetime
import pickle as pkl

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import LinearRegression,  Ridge
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Experiment

from azureml.core.webservice import AciWebservice, AksWebservice, Webservice
from azureml.core.model import Model, InferenceConfig
from azureml.core.image import Image, ContainerImage
from azureml.core.conda_dependencies import CondaDependencies


WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/runtime/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/runtime/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



In [2]:
#create environment file
fraudenv = CondaDependencies()
fraudenv.add_conda_package("scikit-learn")
fraudenv.add_conda_package("numpy")
fraudenv.add_conda_package("scipy")
fraudenv.add_conda_package("pandas")
fraudenv.add_conda_package("py-xgboost<=0.80")
fraudenv.add_pip_package("azureml-defaults")
fraudenv.add_pip_package("azureml-train-automl==1.0.55")
fraudenv.add_pip_package("azureml-core==1.0.55")
fraudenv.add_pip_package("inference-schema")
fraudenv.conda_dependencies_file_path="data/env_dependencies.json"
with open("riskpredictionenv.yml","w") as f:
    f.write(fraudenv.serialize_to_string())

"""
X_df = pd.read_csv('./data/X_train.csv')
Y_df = pd.read_csv('./data/Y_train.csv')
Xtest_df = pd.read_csv('./data/X_test.csv')
Ytest_df = pd.read_csv('./data/Y_test.csv')

Y_df['label'].unique()
Ytest_df['label'].unique()

X_train, X_test,  y_train, y_test = X_df, Xtest_df, Y_df, Ytest_df

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.3,
                                                    random_state=100)

X_valid, X_test, y_valid, y_test = train_test_split(X_valid,
                                                    y_valid,
                                                    test_size=0.3,
                                                    random_state=100)
X_train = pd.DataFrame(X_train)
X_valid = pd.DataFrame(X_valid)
X_test = pd.DataFrame(X_test)

print("The lenght of the training set is:",len(X_train))
print("The lenght of the validating set is:",len(X_valid))
print("The lenght of the testing set is:",len(X_test))
"""



'''
# Substitute the actual version number in the environment file.
# This is not strictly needed in this notebook because the model should have been generated using the current SDK version.
# However, we include this in case this code is used on an experiment from a previous SDK version.

with open(conda_env_file_name, 'r') as cefr:
    content = cefr.read()

with open(conda_env_file_name, 'w') as cefw:
    cefw.write(content.replace(azureml.core.VERSION, dependencies['azureml-sdk']))

# Substitute the actual model id in the script file.

script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()

with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('AutoML29f0a1989best', local_run.model_id))
    '''

In [3]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

input_sample = pd.DataFrame(data=[{"TOTAL_PRICE":869.92,"THIRD_PARTY_ID_SCORE":415,"IS_EXISTING_CUSTOMER":"Y","NUM_PORTIN":0,"FIRST_PARTY_ID_SCORE":356,"ONETIMECHARGE":33.33,"INSTALLMENT_AMOUNT":33.33,"DEVICE_AT_HOME":"Y","FRAUDNET_SCORE":200.0,"NUM_BYOD":0,"IDA_RESULT":"GREEN","EXTERNAL_CREDIT_CHECK_DONE":"Y","SALES_CHANNEL":"ONLINE","MODEL1":"iPhone XR","MODEL2":"NA","MODEL3":"NA","MODEL4":"NA","MODEL5":"NA"}])
output_sample = np.array([0])


def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = 'riskprediction')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        if result ==0:
            result = 'GREEN'
            print('GREEN')
        elif result ==1:
            result = 'YELLOW'
            print('YELLOW')
        else:
            result = 'RED'
            print('RED')
        print('result') 
        fraud_status =[]
        fraud_status.append(result)
        return fraud_status
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result": result.tolist()})


Overwriting score.py


In [4]:
ws = Workspace.from_config()


In [5]:
print(ws)

Workspace.create(name='xm-ml-workspace', subscription_id='be564fde-136b-4709-b7b6-abfc0bdfc134', resource_group='xm-ml')


In [ ]:
# creating a container image
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="riskpredictionenv.yml",
                                 tags = {"key": "1"},
                                 description = "riskpredictionanalysis")
image = Image.create(name = "riskprediction",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

In [6]:
#registering the model
model = Model.register(model_name = "riskprediction11",
                       model_path = "data/riskprediction.pkl",
                       tags = {"key": "1","fraud": "riskpredict"},
                       description = "riskpredictionanalysis",
                       workspace = ws)

print(model)

Registering model riskprediction11
Model(workspace=Workspace.create(name='xm-ml-workspace', subscription_id='be564fde-136b-4709-b7b6-abfc0bdfc134', resource_group='xm-ml'), name=riskprediction11, id=riskprediction11:1, version=1, tags={'key': '1', 'fraud': 'riskpredict'}, properties={})


In [8]:
inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="riskpredictionenv.yml")

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "test-cluster" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating..................................................................................................................

In [12]:
#service deployment
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.image import Image, ContainerImage 

# Set configuration and service name
prod_webservice_name = "fraudclassification-test"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights=True )
# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = ws, 
                                               name = prod_webservice_name,
                                               image = image,
                                       deployment_config = deployment_config,
                                       deployment_target = aks_target)
prod_webservice.wait_for_deployment(show_output = True)

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '361', 'Content-Type': 'application/json', 'Expires': '-1', 'x-ms-correlation-request-id': '01223d4f-204a-48ba-8786-71f5db603f25', 'x-ms-ratelimit-remaining-subscription-writes': '1199', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '7f43449a-3e25-4637-9d9a-129dcf601d24', 'x-ms-client-session-id': '19ee9c91-34c5-45f5-b4b7-cefbf626ac5f', 'x-ms-request-id': '|dfd955dc1ce6ef4ba589d0a57e145a4e.22e3ad93_1.44a28c8f_', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'CENTRALUS:20191024T201149Z:01223d4f-204a-48ba-8786-71f5db603f25', 'X-Content-Type-Options': 'nosniff', 'Date': 'Thu, 24 Oct 2019 20:11:49 GMT'}
Content: b'{"code":"BadRequest","message":"The request is invalid","error":{"code":"BadReq

In [70]:
print("The score uri:",service.scoring_uri)
print("The swagger:", service.swagger_uri)
primary, secondary = service.get_keys()
print("the primary key:",primary)
print("the secondry key:",secondary)

The score uri: https://riskpredictionmyrtmu.centralus.cloudapp.azure.com:443/api/v1/service/riskpredictionanalysis1/score
The swagger: https://riskpredictionmyrtmu.centralus.cloudapp.azure.com:443/api/v1/service/riskpredictionanalysis1/swagger.json
the primary key: FqBQF5j9bhBI3YDzE0AcAFflJEfnxeU1
the secondry key: SqTeOZZlgVwNYttGCnaqDyP1o5Lxxqfg


In [ ]:
print(service.get_logs())

# Alla : testing new stuff, please don't change

In [81]:
package = Model.package(ws, [model], inference_config, generate_dockerfile=True)
package.wait_for_creation(show_output=True)
# Download the package.
package.save("./imagefiles")
# Get the Azure container registry that the model/Dockerfile uses.
acr=package.get_container_registry()
print("Address:", acr.address)
print("Username:", acr.username)
print("Password:", acr.password)


'''
Package creation Succeeded
Logging into Docker registry xmmlworkspac13b1f937.azurecr.io
Address: xmmlworkspac13b1f937.azurecr.io
Username: xmmlworkspac13b1f937
Password: ++fbZmXo0xdhN/zwwUSM5pKiPj=8TXsA
docker login <xmmlworkspac13b1f937.azurecr.io> -u <xmmlworkspac13b1f937> -p <++fbZmXo0xdhN/zwwUSM5pKiPj=8TXsA>

'''

Package creation Succeeded
Logging into Docker registry xmmlworkspac13b1f937.azurecr.io
Address: xmmlworkspac13b1f937.azurecr.io
Username: xmmlworkspac13b1f937
Password: ++fbZmXo0xdhN/zwwUSM5pKiPj=8TXsA


In [82]:
import requests
import json

# URL for the web service.
scoring_uri = 'http://localhost:6789/score'

# Two sets of data to score, so we get two results back.
data = {"data":
        [
            [ 1,2,3,4,5,6,7,8,9,10 ],
            [ 10,9,8,7,6,5,4,3,2,1 ]
        ]
        }
# Convert to JSON string.
input_data = json.dumps(data)

# Set the content type.
headers = {'Content-Type': 'application/json'}

# Make the request and display the response.
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7e0e4015f8>: Failed to establish a new connection: [Errno 111] Connection refused',))